In [2]:
import pandas as pd
from pandas.io import gbq
import numpy as np
import os
import glob
import re

In [3]:
import pydata_google_auth

In [4]:
    connect_bq = [
        'https://www.googleapis.com/auth/cloud-platform',
        'https://www.googleapis.com/auth/drive',
    ]

    credentials = pydata_google_auth.get_user_credentials(
        connect_bq,
        auth_local_webserver=True,
    )

#### Query that retrieves add_ons per user for a month and the add ons that a bundle has

In [5]:
sql_query = """
CREATE TEMP FUNCTION STRING_DEDUP(str STRING) AS (
(SELECT STRING_AGG(item ORDER BY item) FROM (
SELECT DISTINCT item FROM UNNEST(SPLIT(str)) item 
)) 
);
----------------------------------------------------------------------------------------------------------------------
with daily_status as (
Select * , DATE_TRUNC(day, month) as change_month
from `fubotv-prod.data_insights.daily_status_static_update` t1
where final_status_restated like '%paid%'
and day >= '2021-01-01'
AND day <= '2022-05-31'
),
add_on as (
select account_code, change_month, STRING_DEDUP(add_ons) as addons_sorted,STRING_DEDUP(bundle_add_on_list) as bundle_addons_sorted
FROM daily_status
)
select DISTINCT change_month as month, account_code, bundle_addons_sorted, addons_sorted
from add_on
"""

In [6]:
df = pd.read_gbq(
    sql_query,
    project_id='fubotv-prod',
    credentials=credentials,
)

#### Checking / Housekeeping

In [31]:
df.head()

,month,nonbundle_addons,account_code
0,2021-01-01,advanced-dvr,11215
1,2021-01-01,advanced-dvr-1000,3508
2,2021-01-01,advanced-dvr-250,56869
3,2021-01-01,adventure,6088
4,2021-01-01,amc-premiere,2725


##### if the bundles addon column is empty, discard that row

In [8]:
df = df.dropna()

##### Comparing bundle add_ons to the add_ons field and identifying the difference between these two fields and joining each add_on by a comma to a new field called new_addition

In [10]:
df['nonbundle_addons'] = [','.join(set(n.split(',')) - set(o.split(','))) 
                                                          for o, n in zip(df.bundle_addons_sorted, df.addons_sorted)]

In [11]:
df

,month,account_code,bundle_addons_sorted,addons_sorted,nonbundle_addons
0,2021-03-01,5d7fb49379bf5f000174717d,"advanced-dvr,third-screen","advanced-dvr,intl-sports-plus,unlimited-screen...","intl-sports-plus,unlimited-screen-home"
1,2021-08-01,5b46489d1467cf4b8fe2a265,third-screen,third-screen,
2,2022-05-01,6286e6b418025800015d35b6,"4k-screen,advanced-dvr-1000,fubo-extra-lite,ne...","4k-screen,advanced-dvr-1000,fubo-extra-lite,ne...",rsn-fee
3,2021-09-01,5f5c21d8e10f990001894919,"advanced-dvr,third-screen",third-screen,
5,2021-12-01,57379de62ab90001002a6271,"advanced-dvr,third-screen","advanced-dvr,fubo-extra-lite,intl-sports-plus,...","intl-sports-plus,fubo-extra-lite,unlimited-scr..."
...,...,...,...,...,...
17337684,2022-05-01,6144ba5328e0760001a9a6a4,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee
17337685,2022-04-01,5d9fdbc2374be00001c00ea9,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee
17337686,2021-08-01,5fee74dd3b75a50001ab9709,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee
17337687,2022-05-01,6167b1cd129abf00011122c1,"advanced-dvr-1000,third-screen,unlimited-scree...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...","fubo-extra-lite,rsn-fee"


#### Splitting new_addition column into additional columns and exploding them into individual rows

In [12]:
df['nonbundle_addons'] = df['nonbundle_addons'].str.split(',')
df = df.explode('nonbundle_addons').reset_index(drop=True)
cols = list(df.columns)
df = df[cols]

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17748722 entries, 0 to 17748721
Data columns (total 5 columns):
 #   Column                Dtype         
---  ------                -----         
 0   month                 datetime64[ns]
 1   account_code          object        
 2   bundle_addons_sorted  object        
 3   addons_sorted         object        
 4   nonbundle_addons      object        
dtypes: datetime64[ns](1), object(4)
memory usage: 677.1+ MB


In [20]:
df = df.drop(['addons_sorted', 'bundle_addons_sorted'], axis=1)

,month,account_code,bundle_addons_sorted,addons_sorted,nonbundle_addons
0,2021-03-01,5d7fb49379bf5f000174717d,"advanced-dvr,third-screen","advanced-dvr,intl-sports-plus,unlimited-screen...",intl-sports-plus
1,2021-03-01,5d7fb49379bf5f000174717d,"advanced-dvr,third-screen","advanced-dvr,intl-sports-plus,unlimited-screen...",unlimited-screen-home
5,2021-12-01,57379de62ab90001002a6271,"advanced-dvr,third-screen","advanced-dvr,fubo-extra-lite,intl-sports-plus,...",intl-sports-plus
6,2021-12-01,57379de62ab90001002a6271,"advanced-dvr,third-screen","advanced-dvr,fubo-extra-lite,intl-sports-plus,...",fubo-extra-lite
7,2021-12-01,57379de62ab90001002a6271,"advanced-dvr,third-screen","advanced-dvr,fubo-extra-lite,intl-sports-plus,...",unlimited-screen-home


In [21]:
df = df.drop_duplicates()

In [14]:
print(df)

              month              account_code  \
0        2021-03-01  5d7fb49379bf5f000174717d   
1        2021-03-01  5d7fb49379bf5f000174717d   
2        2021-08-01  5b46489d1467cf4b8fe2a265   
3        2022-05-01  6286e6b418025800015d35b6   
4        2021-09-01  5f5c21d8e10f990001894919   
...             ...                       ...   
17748717 2022-04-01  5d9fdbc2374be00001c00ea9   
17748718 2021-08-01  5fee74dd3b75a50001ab9709   
17748719 2022-05-01  6167b1cd129abf00011122c1   
17748720 2022-05-01  6167b1cd129abf00011122c1   
17748721 2021-06-01  5de2d9821d279d00018f60cb   

                                       bundle_addons_sorted  \
0                                 advanced-dvr,third-screen   
1                                 advanced-dvr,third-screen   
2                                              third-screen   
3         4k-screen,advanced-dvr-1000,fubo-extra-lite,ne...   
4                                 advanced-dvr,third-screen   
...                              

In [15]:
df

,month,account_code,bundle_addons_sorted,addons_sorted,nonbundle_addons
0,2021-03-01,5d7fb49379bf5f000174717d,"advanced-dvr,third-screen","advanced-dvr,intl-sports-plus,unlimited-screen...",intl-sports-plus
1,2021-03-01,5d7fb49379bf5f000174717d,"advanced-dvr,third-screen","advanced-dvr,intl-sports-plus,unlimited-screen...",unlimited-screen-home
2,2021-08-01,5b46489d1467cf4b8fe2a265,third-screen,third-screen,
3,2022-05-01,6286e6b418025800015d35b6,"4k-screen,advanced-dvr-1000,fubo-extra-lite,ne...","4k-screen,advanced-dvr-1000,fubo-extra-lite,ne...",rsn-fee
4,2021-09-01,5f5c21d8e10f990001894919,"advanced-dvr,third-screen",third-screen,
...,...,...,...,...,...
17748717,2022-04-01,5d9fdbc2374be00001c00ea9,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee
17748718,2021-08-01,5fee74dd3b75a50001ab9709,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee
17748719,2022-05-01,6167b1cd129abf00011122c1,"advanced-dvr-1000,third-screen,unlimited-scree...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",fubo-extra-lite
17748720,2022-05-01,6167b1cd129abf00011122c1,"advanced-dvr-1000,third-screen,unlimited-scree...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee


###### Remove RSN fee and blanks from the field

In [16]:
df = df[df.nonbundle_addons != 'rsn-fee']
df = df[df.nonbundle_addons != '']
df = df[df.nonbundle_addons != ""]

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522 entries, 1 to 557
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   month             522 non-null    datetime64[ns]
 1   nonbundle_addons  522 non-null    object        
 2   account_code      522 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 16.3+ KB


In [22]:
df.head()

,month,account_code,nonbundle_addons
0,2021-03-01,5d7fb49379bf5f000174717d,intl-sports-plus
1,2021-03-01,5d7fb49379bf5f000174717d,unlimited-screen-home
5,2021-12-01,57379de62ab90001002a6271,intl-sports-plus
6,2021-12-01,57379de62ab90001002a6271,fubo-extra-lite
7,2021-12-01,57379de62ab90001002a6271,unlimited-screen-home


###### COnverting timestamp to Date

In [25]:
df['month'] = pd.to_datetime(df['month'], format="%Y/%m/%d").dt.date

In [26]:
df.dtypes

month               object
account_code        object
nonbundle_addons    object
dtype: object

#### Account code level Table

In [28]:
df.to_gbq ( destination_table='business_analytics.temp_nonbundle_addon_account_code', project_id= 'fubotv-dev', if_exists= 'replace', table_schema = [{'name':'month','type': 'DATE'}])

#### Aggregating by month and new_addition to count distinct account_code

In [29]:
df = df.groupby(['month', 'nonbundle_addons'],  as_index=False ).agg({'account_code': 'nunique'})

In [30]:
df.to_gbq ( destination_table='business_analytics.temp_nonbundle_addon_counts', project_id= 'fubotv-dev', if_exists= 'replace', table_schema = [{'name':'month','type': 'DATE'}])